Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

C:\Users\fabio\Anaconda3\envs\tflw\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = r'Z:\Staff\Fabio_Disidoro\Hip_hop\Programming\MachineLearning\Udacity_Course\notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, verbose = True):
    # stride list = [batch,height,width,channels]
    # output of 2D convolution is batch_size, output height, output width, depth
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME') 
    if verbose: print("First 2D convolution", conv)
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    if verbose: print("Second 2D convolution", conv)
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

First 2D convolution Tensor("Conv2D:0", shape=(16, 14, 14, 16), dtype=float32)
Second 2D convolution Tensor("Conv2D_1:0", shape=(16, 7, 7, 16), dtype=float32)
First 2D convolution Tensor("Conv2D_2:0", shape=(10000, 14, 14, 16), dtype=float32)
Second 2D convolution Tensor("Conv2D_3:0", shape=(10000, 7, 7, 16), dtype=float32)
First 2D convolution Tensor("Conv2D_4:0", shape=(10000, 14, 14, 16), dtype=float32)
Second 2D convolution Tensor("Conv2D_5:0", shape=(10000, 7, 7, 16), dtype=float32)


In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.854214
Minibatch accuracy: 12.5%
Validation accuracy: 11.6%
Minibatch loss at step 50: 1.388513
Minibatch accuracy: 68.8%
Validation accuracy: 57.7%
Minibatch loss at step 100: 1.009117
Minibatch accuracy: 62.5%
Validation accuracy: 66.0%
Minibatch loss at step 150: 0.766785
Minibatch accuracy: 81.2%
Validation accuracy: 77.4%
Minibatch loss at step 200: 1.123719
Minibatch accuracy: 75.0%
Validation accuracy: 78.2%
Minibatch loss at step 250: 0.821010
Minibatch accuracy: 75.0%
Validation accuracy: 77.0%
Minibatch loss at step 300: 1.269127
Minibatch accuracy: 62.5%
Validation accuracy: 78.4%
Minibatch loss at step 350: 1.483872
Minibatch accuracy: 62.5%
Validation accuracy: 78.5%
Minibatch loss at step 400: 0.807521
Minibatch accuracy: 68.8%
Validation accuracy: 79.9%
Minibatch loss at step 450: 0.765928
Minibatch accuracy: 75.0%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.452898
Minibatch accuracy: 93.8%
Validation accuracy: 80.9%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [16]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, verbose = True):
    # stride list = [batch,height,width,channels]
    # output of 2D convolution is batch_size, output height, output width, depth
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME') 
    if verbose: print("First 2D convolution ", conv)
    reduced = tf.nn.max_pool(conv,[1, 2, 2, 1],[1, 2, 2, 1], padding='SAME')
    if verbose: print("Max pooling 1 ", reduced)
    hidden = tf.nn.relu(reduced + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    if verbose: print("Second 2D convolution", conv)
    reduced = tf.nn.max_pool(conv,[1, 2, 2, 1],[1, 2, 2, 1], padding='SAME')
    if verbose: print("Max pooling 2 ", reduced)
    hidden = tf.nn.relu(reduced + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    print(reshape)
    print(layer3_weights)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

First 2D convolution  Tensor("Conv2D:0", shape=(16, 28, 28, 16), dtype=float32)
Max pooling 1  Tensor("MaxPool:0", shape=(16, 14, 14, 16), dtype=float32)
Second 2D convolution Tensor("Conv2D_1:0", shape=(16, 14, 14, 16), dtype=float32)
Max pooling 2  Tensor("MaxPool_1:0", shape=(16, 7, 7, 16), dtype=float32)
Tensor("Reshape:0", shape=(16, 784), dtype=float32)
<tf.Variable 'Variable_4:0' shape=(784, 64) dtype=float32_ref>
First 2D convolution  Tensor("Conv2D_2:0", shape=(10000, 28, 28, 16), dtype=float32)
Max pooling 1  Tensor("MaxPool_2:0", shape=(10000, 14, 14, 16), dtype=float32)
Second 2D convolution Tensor("Conv2D_3:0", shape=(10000, 14, 14, 16), dtype=float32)
Max pooling 2  Tensor("MaxPool_3:0", shape=(10000, 7, 7, 16), dtype=float32)
Tensor("Reshape_1:0", shape=(10000, 784), dtype=float32)
<tf.Variable 'Variable_4:0' shape=(784, 64) dtype=float32_ref>
First 2D convolution  Tensor("Conv2D_4:0", shape=(10000, 28, 28, 16), dtype=float32)
Max pooling 1  Tensor("MaxPool_4:0", shape=(

In [17]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.711830
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.927758
Minibatch accuracy: 37.5%
Validation accuracy: 25.7%
Minibatch loss at step 100: 1.409674
Minibatch accuracy: 56.2%
Validation accuracy: 52.6%
Minibatch loss at step 150: 0.925331
Minibatch accuracy: 81.2%
Validation accuracy: 74.0%
Minibatch loss at step 200: 0.993364
Minibatch accuracy: 75.0%
Validation accuracy: 76.9%
Minibatch loss at step 250: 0.709587
Minibatch accuracy: 81.2%
Validation accuracy: 78.8%
Minibatch loss at step 300: 1.024016
Minibatch accuracy: 75.0%
Validation accuracy: 79.6%
Minibatch loss at step 350: 1.186778
Minibatch accuracy: 62.5%
Validation accuracy: 79.9%
Minibatch loss at step 400: 0.928263
Minibatch accuracy: 75.0%
Validation accuracy: 82.2%
Minibatch loss at step 450: 0.734685
Minibatch accuracy: 81.2%
Validation accuracy: 82.7%
Minibatch loss at step 500: 0.262428
Minibatch accuracy: 93.8%
Validation accuracy: 82.3%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

### LeNet5 implementation

In [5]:
# Pad images with 0s (LenNet5 requires 32X32X1 images instead of 28X28X1) 
X_train      = np.pad(train_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_validation = np.pad(valid_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
X_test       = np.pad(test_dataset, ((0,0),(2,2),(2,2),(0,0)), 'constant')
    
print("Updated Image Shape: {}".format(X_train[0].shape))

Updated Image Shape: (32, 32, 1)


In [13]:
epochs = 10
batch_size = 128
image_size = 32
num_channels = 1
mu = 0
sigma = 0.1
depth_conv1 = 6
depth_conv2 = 16
depth_fc1 = 120
depth_fc2 = 84
depth_fc3 = 10
patch_size = 5
 

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(X_validation)
  tf_test_dataset = tf.constant(X_test)
  
  # Variables.
  conv1_w = tf.Variable(tf.truncated_normal(shape = [patch_size, patch_size, num_channels, depth_conv1],
                                            mean = mu, stddev = sigma))
  conv1_b = tf.Variable(tf.zeros(depth_conv1))
  conv2_w = tf.Variable(tf.truncated_normal(shape = [patch_size, patch_size, depth_conv1, depth_conv2],
                                            mean = mu, stddev = sigma))
  conv2_b = tf.Variable(tf.zeros(depth_conv2))
  fc1_w = tf.Variable(tf.truncated_normal(shape = (5*5*16,depth_fc1), mean = mu, stddev = sigma))
  fc1_b = tf.Variable(tf.zeros(depth_fc1))
  fc2_w = tf.Variable(tf.truncated_normal(shape = (depth_fc1, depth_fc2), mean = mu, stddev = sigma))
  fc2_b = tf.Variable(tf.zeros(depth_fc2))
  fc3_w = tf.Variable(tf.truncated_normal(shape = (depth_fc2, depth_fc3), mean = mu, stddev = sigma))
  fc3_b = tf.Variable(tf.zeros(depth_fc3))

  # Model.
  def LeNet5(data):
    # stride list = [batch,height,width,channels]
    # output of 2D convolution is batch_size, output height, output width, depth
    
    # Layer 1: Convolutional. Input = 32x32x1. Output = 28x28x6 (reduced by 4 due to valid padding with patch size of 5)
    conv1 = tf.nn.conv2d(data, conv1_w, strides = [1, 1, 1, 1], padding='VALID') + conv1_b
    conv1 = tf.nn.relu(conv1)
    
    # Pooling 1. Input = 28x28x6. Output = 14x14x6.
    pool_1 = tf.nn.max_pool(conv1,ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'VALID')
    
    # Layer 2: Convolutional. Input = 14x14x6. Output = 10x10x16. (reduced by 4 due to valid padding with patch size of 5)
    conv2 = tf.nn.conv2d(pool_1, conv2_w, [1, 1, 1, 1], padding='VALID') + conv2_b
    conv2 = tf.nn.relu(conv2)
    
    # Pooling 2. Input = 10x10x16. Output = 5x5x16.
    pool_2 = tf.nn.max_pool(conv2,ksize = [1, 2, 2, 1], strides =  [1, 2, 2, 1], padding='VALID')
    
    # Flatten to one-dim array of size 5x5x16 = 400.
    shape = pool_2.get_shape().as_list()
    flattened = tf.reshape(pool_2, [shape[0], shape[1] * shape[2] * shape[3]])
    #print(flattened)
    
    # Layer 3: Fully Connected. Input = 400. Output = 120.
    fc1 = tf.matmul(flattened, fc1_w) + fc1_b
    fc1 = tf.nn.relu(fc1)
    
    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fc2 = tf.matmul(fc1, fc2_w) + fc2_b
    fc2 = tf.nn.relu(fc2)
    
    # Layer 5: Fully Connected. Input = 84. Output = 10.
    fc3 = tf.matmul(fc2, fc3_w) + fc3_b
    logits = tf.nn.relu(fc3)
    
    return logits
  
  # Training computation.
  logits = LeNet5(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  #optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(LeNet5(tf_valid_dataset))
  test_prediction = tf.nn.softmax(LeNet5(tf_test_dataset))

In [14]:
num_steps = 4001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = X_train[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.300358
Minibatch accuracy: 16.4%
Validation accuracy: 14.1%
Minibatch loss at step 50: 1.785339
Minibatch accuracy: 39.1%
Validation accuracy: 45.4%
Minibatch loss at step 100: 1.534113
Minibatch accuracy: 47.7%
Validation accuracy: 56.6%
Minibatch loss at step 150: 0.957065
Minibatch accuracy: 71.1%
Validation accuracy: 71.2%
Minibatch loss at step 200: 0.921855
Minibatch accuracy: 68.8%
Validation accuracy: 72.9%
Minibatch loss at step 250: 0.828875
Minibatch accuracy: 73.4%
Validation accuracy: 72.0%
Minibatch loss at step 300: 0.861899
Minibatch accuracy: 71.1%
Validation accuracy: 75.2%
Minibatch loss at step 350: 0.600885
Minibatch accuracy: 79.7%
Validation accuracy: 76.1%
Minibatch loss at step 400: 0.783065
Minibatch accuracy: 77.3%
Validation accuracy: 76.5%
Minibatch loss at step 450: 0.680143
Minibatch accuracy: 76.6%
Validation accuracy: 76.4%
Minibatch loss at step 500: 0.634300
Minibatch accuracy: 78.9%
Validation accuracy: 77.2%
M